In [1]:
import os
import gzip
import shutil
import nibabel as nib
import numpy as np
import cv2
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'components')))
from modules import *
import yaml
from train_yolov8 import *

In [4]:
"""
This function performs the processing and removal of files located in a designated input directory. 
The function navigates through the directory hierarchy to find files with a `.nii.gz` extension, uncompresses these files
to their `.nii` format by removing the `.gz` compression, and then removes the original `.nii.gz` files to conserve storage space. 
The method includes confirmation messages for both the decompression and deletion processes, guaranteeing that the actions are recorded. 
"""
def process_and_cleanup_files(input_path):
    for root, dirs, files in os.walk(input_path):
        root = root.replace("\\", "/")
        for file in files:
            if file.endswith('.nii.gz'):
                nii_gz_file_path = os.path.join(root, file)
                decompressed_file_path = os.path.splitext(nii_gz_file_path)[0]  # Remove .gz extension
                # Decompress the .nii.gz file
                with gzip.open(nii_gz_file_path, 'rb') as f_in:
                    with open(decompressed_file_path, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                # Print confirmation
                print(f'\nDecompressed {nii_gz_file_path} to {decompressed_file_path}')
                # Delete the original .nii.gz file
                os.remove(nii_gz_file_path)
                print(f'\nDeleted {nii_gz_file_path}')
                print("="*100)
# Define your input path here
input_path = '../Inputs/organs_masks'
# Process and cleanup the files
process_and_cleanup_files(input_path)


Decompressed ../Inputs/organs_masks/10217/16066\colon.nii.gz to ../Inputs/organs_masks/10217/16066\colon.nii

Deleted ../Inputs/organs_masks/10217/16066\colon.nii.gz

Decompressed ../Inputs/organs_masks/10217/16066\kidney_left.nii.gz to ../Inputs/organs_masks/10217/16066\kidney_left.nii

Deleted ../Inputs/organs_masks/10217/16066\kidney_left.nii.gz

Decompressed ../Inputs/organs_masks/10217/16066\kidney_right.nii.gz to ../Inputs/organs_masks/10217/16066\kidney_right.nii

Deleted ../Inputs/organs_masks/10217/16066\kidney_right.nii.gz

Decompressed ../Inputs/organs_masks/10217/16066\liver.nii.gz to ../Inputs/organs_masks/10217/16066\liver.nii

Deleted ../Inputs/organs_masks/10217/16066\liver.nii.gz

Decompressed ../Inputs/organs_masks/10217/16066\small_bowel.nii.gz to ../Inputs/organs_masks/10217/16066\small_bowel.nii

Deleted ../Inputs/organs_masks/10217/16066\small_bowel.nii.gz

Decompressed ../Inputs/organs_masks/10217/16066\spleen.nii.gz to ../Inputs/organs_masks/10217/16066\spleen.

In [5]:
"""
This function reorganizes files within the specified input directory (`'../Inputs/organs_masks'`) by moving and renaming `.nii` files based on 
the organ name, patient ID, and series ID extracted from the directory structure. 
The files are moved into organ-specific folders, and the directory structure is cleaned up by removing empty folders after the reorganization. 
"""
def reorganize_files(input_path):
    # Keep track of the folders to potentially remove
    folders_to_remove = set()
    for root, dirs, files in os.walk(input_path, topdown=False):
        root = root.replace("\\", "/")
        # Extract the patient_id and series_id from the root path
        parts = root.split('/')
        if len(parts) < 3:
            continue
        patient_id = parts[-2]
        series_id = parts[-1]
        for file in files:
            if file.endswith('.nii'):
                # Extract the organ name from the file name
                organ_name = os.path.splitext(file)[0]
                # Define the new file name and path
                new_folder = os.path.join(input_path, organ_name)
                new_file_name = f"{organ_name}_{patient_id}_{series_id}.nii"
                new_file_path = os.path.join(new_folder, new_file_name)
                # Create the new directory for the organ if it doesn't exist
                if not os.path.exists(new_folder):
                    os.makedirs(new_folder)
                # Move and rename the file
                old_file_path = os.path.join(root, file)
                os.rename(old_file_path, new_file_path)
                print(f"Moved {old_file_path} to {new_file_path}")
                # Add all directories from the current root to the patient_id level to the removal set
                current_dir = root
                while len(current_dir.split('/')) > len(parts) - 2:
                    folders_to_remove.add(current_dir)
                    parent_dir = os.path.dirname(current_dir)
                    if parent_dir == current_dir:
                        break
                    current_dir = parent_dir
    # Remove old folders up to the patient_id level if they are empty
    for folder in sorted(folders_to_remove, reverse=True):
        if not os.listdir(folder):
            shutil.rmtree(folder)
            print(f"Removed empty folder {folder}")
# Define your input path here
input_path = '../Inputs/organs_masks'
# Reorganize the files
reorganize_files(input_path)

Moved ../Inputs/organs_masks/10217/16066\colon.nii to ../Inputs/organs_masks\colon\colon_10217_16066.nii
Moved ../Inputs/organs_masks/10217/16066\kidney_left.nii to ../Inputs/organs_masks\kidney_left\kidney_left_10217_16066.nii
Moved ../Inputs/organs_masks/10217/16066\kidney_right.nii to ../Inputs/organs_masks\kidney_right\kidney_right_10217_16066.nii
Moved ../Inputs/organs_masks/10217/16066\liver.nii to ../Inputs/organs_masks\liver\liver_10217_16066.nii
Moved ../Inputs/organs_masks/10217/16066\small_bowel.nii to ../Inputs/organs_masks\small_bowel\small_bowel_10217_16066.nii
Moved ../Inputs/organs_masks/10217/16066\spleen.nii to ../Inputs/organs_masks\spleen\spleen_10217_16066.nii
Moved ../Inputs/organs_masks/10228/40471\colon.nii to ../Inputs/organs_masks\colon\colon_10228_40471.nii
Moved ../Inputs/organs_masks/10228/40471\kidney_left.nii to ../Inputs/organs_masks\kidney_left\kidney_left_10228_40471.nii
Moved ../Inputs/organs_masks/10228/40471\kidney_right.nii to ../Inputs/organs_mask

In [6]:
"""
The function efficiently arranges and modifies files in provided source directories by relocating them to a selected destination directory 
and renaming them based on the category of the organ. To address possible conflicts in filenames, it resolves the issue by adding a numerical 
suffix to the filename. Upon relocating the files, the script verifies and eliminates any source folders that are devoid of content. 
"""
def move_and_rename_files(src_dirs, dest_dir, organ_type):
    # Create the destination directory if it doesn't exist
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    for src_dir in src_dirs:
        print(f"Processing source directory: {src_dir}")
        src_dir = src_dir.replace('\\', '/')
        for filename in os.listdir(src_dir):
            src_file = os.path.join(src_dir, filename)
            if os.path.isfile(src_file):
                # Extract patient_id and series_id from the filename
                parts = filename.split('_')
                patient_id = parts[-2]
                series_id = parts[-1]
                # Define the new filename
                new_filename = f"{organ_type}_{patient_id}_{series_id}"
                new_file_path = os.path.join(dest_dir, new_filename)
                print(f"New filename: {new_filename}")
                print(f"New file path: {new_file_path}")
                # Handle filename collisions by adding an integer suffix
                if os.path.exists(new_file_path):
                    base_name, ext = os.path.splitext(new_filename)
                    counter = 0
                    while os.path.exists(new_file_path):
                        new_filename = f"{base_name}_{counter}{ext}"
                        new_file_path = os.path.join(dest_dir, new_filename)
                        counter += 1
                # Move and rename the file
                shutil.move(src_file, new_file_path)
    print(f"Files moved and renamed successfully to {dest_dir}!")
    # Remove empty source directories
    for src_dir in src_dirs:
        src_dir = src_dir.replace('\\', '/')
        if not os.listdir(src_dir):  # Check if the directory is empty
            os.rmdir(src_dir)
            print(f"Removed empty source directory: {src_dir}")
# Define the input path
input_path = '../Inputs/organs_masks'
# Define source and destination directories for kidney
kidney_src_dirs = [os.path.join(input_path, dir) for dir in ['kidney_right', 'kidney_left']]
kidney_dest_dir = os.path.join(input_path, 'kidney')
# Define source and destination directories for bowel
bowel_src_dirs = [os.path.join(input_path, dir) for dir in ['colon', 'small_bowel']]
bowel_dest_dir = os.path.join(input_path, 'bowel')
# Move and rename files for kidney
move_and_rename_files(kidney_src_dirs, kidney_dest_dir, 'kidney')
# Move and rename files for bowel
move_and_rename_files(bowel_src_dirs, bowel_dest_dir, 'bowel')

Processing source directory: ../Inputs/organs_masks\kidney_right
New filename: kidney_10217_16066.nii
New file path: ../Inputs/organs_masks\kidney\kidney_10217_16066.nii
New filename: kidney_10228_40471.nii
New file path: ../Inputs/organs_masks\kidney\kidney_10228_40471.nii
New filename: kidney_10300_31085.nii
New file path: ../Inputs/organs_masks\kidney\kidney_10300_31085.nii
New filename: kidney_10937_53000.nii
New file path: ../Inputs/organs_masks\kidney\kidney_10937_53000.nii
New filename: kidney_11177_13041.nii
New file path: ../Inputs/organs_masks\kidney\kidney_11177_13041.nii
New filename: kidney_13623_24645.nii
New file path: ../Inputs/organs_masks\kidney\kidney_13623_24645.nii
New filename: kidney_13741_42173.nii
New file path: ../Inputs/organs_masks\kidney\kidney_13741_42173.nii
New filename: kidney_13741_6172.nii
New file path: ../Inputs/organs_masks\kidney\kidney_13741_6172.nii
New filename: kidney_13794_24774.nii
New file path: ../Inputs/organs_masks\kidney\kidney_13794_24

In [7]:
"""
Function utilises NIfTI files to build masks and accompanying annotations for various organs (such as liver, spleen, colon, and kidney). 
These masks and annotations are specifically designed for training object recognition models like YOLO. The process starts by establishing 
the required folders and then proceeds to loop over each NIfTI file in the segmentation directory. During this iteration, it extracts pertinent 
identifiers such as `patient_id` and `series_id` from the filenames. The method reads individual picture slices, applies a scaling factor, and 
creates scaled masks for slices that have non-zero pixel values. Contours are identified in order to generate bounding boxes. 
These bounding boxes are further labelled according to the kind of organ and its health condition, using a predetermined mapping 
called `combined_labels`. The output consists of mask pictures and YOLO-format annotations that are stored in standardised folders. 
This makes it easier to create training datasets for deep learning models.
"""
def make_masks_bounding_box_and_annotations(scale_factor, segmentation_dir, mask_dir, annotation_dir, label_df, combined_labels):
    # Ensure the output directories exist
    os.makedirs(mask_dir, exist_ok=True)
    os.makedirs(annotation_dir, exist_ok=True)
    # Derive the organ name from the last value in the segmentation directory path
    organ_name = os.path.basename(segmentation_dir.strip('/\\'))
    # Create organ-specific directories inside mask_dir and annotation_dir
    organ_mask_dir = os.path.join(mask_dir, organ_name)
    organ_annotation_dir = os.path.join(annotation_dir, organ_name)
    os.makedirs(organ_mask_dir, exist_ok=True)
    os.makedirs(organ_annotation_dir, exist_ok=True)
    # Process each NIfTI file in the given segmentation directory
    for nii_file in os.listdir(segmentation_dir):
        if not nii_file.endswith('.nii'):
            print(f"Skipping non-NIfTI file: {nii_file}")
            continue
        nii_path = os.path.join(segmentation_dir, nii_file).replace('\\', '/')
        print(f"\nProcessing file: {nii_path}")
        if not os.path.exists(nii_path):
            print(f"File {nii_file} not found in the directory. Skipping...")
            continue
        print(f".............Processing..........:   {nii_file}")
        # Load the NIfTI image
        nii_img = nib.load(nii_path)
        data = nii_img.get_fdata()
        print(f"Shape of Image: {data.shape}")
        base_filename = os.path.splitext(os.path.basename(nii_path))[0]
        parts = base_filename.split('_')
        if len(parts) < 3:
            print(f"Filename {nii_file} does not conform to expected format. Skipping...")
            continue
        patient_id = parts[1]
        series_id = parts[2]
        fourth_part = parts[3] if len(parts) > 3 else ''  # The fourth part of the filename, if it exists
        # Find the matching row in label_df based on series_id
        filtered_df = label_df[label_df['series_id'].astype(str) == series_id]
        if filtered_df.empty:
            print(f"\nNo matching entry found in label_df for series_id {series_id}. Skipping processing.")
            continue
        # Get the first row (assuming only one entry per series_id)
        label_row = filtered_df.iloc[0]
        # Create a combined ID for naming based on series_id and additional information
        combined_id = f"{label_row['combined_id']}"
        for i in range(data.shape[2]):
            img = data[:, :, i] * scale_factor
            if img.max() > 0:  # Check if the maximum pixel value in the image is greater than 0
                img = np.rot90(img)  # Rotate the image
                # Resize the image 
                resized_img = cv2.resize(img, (512, 512))
                # Initialize a blank mask image
                mask_img = np.zeros_like(resized_img, dtype=np.uint8)
                bboxes = []
                # Load the organ mask image
                organ_nii_path = os.path.join(segmentation_dir, f'{organ_name}_{patient_id}_{series_id}_{fourth_part}.nii') if fourth_part else os.path.join(segmentation_dir, f'{organ_name}_{patient_id}_{series_id}.nii')
                if not os.path.exists(organ_nii_path):
                    print(f"File {organ_nii_path} not found in the directory. Skipping...")
                    continue
                organ_nii_img = nib.load(organ_nii_path)
                organ_data = organ_nii_img.get_fdata()
                organ_img = organ_data[:, :, i] * scale_factor
                if organ_img.max() > 0:
                    organ_img = np.rot90(organ_img)
                    organ_resized_img = cv2.resize(organ_img, (512, 512))
                    mask_img[organ_resized_img > 0] = 255
                    # Find contours and create bounding boxes
                    contours, _ = cv2.findContours((organ_resized_img > 0).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    for contour in contours:
                        x, y, w, h = cv2.boundingRect(contour)
                        bboxes.append((organ_name, x, y, w, h))  # Store organ name directly
                # Save the resized image as a PNG file with combined ID and slice number
                png_filename = os.path.join(organ_mask_dir, f'{combined_id}_{fourth_part}_{i:04d}.png') if fourth_part else os.path.join(organ_mask_dir, f'{combined_id}_{i:04d}.png')
                cv2.imwrite(png_filename, resized_img)
                # Save the mask image if it has content
                if mask_img.max() > 0:
                    mask_filename = os.path.join(organ_mask_dir, f'{combined_id}_{fourth_part}_{i:04d}.png') if fourth_part else os.path.join(organ_mask_dir, f'{combined_id}_{i:04d}.png')
                    cv2.imwrite(mask_filename, mask_img)
                    # Save the bounding boxes in YOLO format with combined ID and slice number
                    annotation_filename = os.path.join(organ_annotation_dir, f'{combined_id}_{fourth_part}_{i:04d}.txt') if fourth_part else os.path.join(organ_annotation_dir, f'{combined_id}_{i:04d}.txt')
                    with open(annotation_filename, 'w') as f:
                        for bbox in bboxes:
                            organ, x, y, w, h = bbox
                            label = organ
                            # Define label based on CSV information
                            label_info = {
                                'liver': ('healthy' if label_row['liver_healthy'] == 1 else
                                          'low' if label_row['liver_low'] == 1 else
                                          'high' if label_row['liver_high'] == 1 else None),
                                'spleen': ('healthy' if label_row['spleen_healthy'] == 1 else
                                           'low' if label_row['spleen_low'] == 1 else
                                           'high' if label_row['spleen_high'] == 1 else None),
                                'bowel': ('healthy' if label_row['bowel_healthy'] == 1 else
                                          'injury' if label_row['bowel_injury'] == 1 else None),
                                'kidney': ('healthy' if label_row['kidney_healthy'] == 1 else
                                           'low' if label_row['kidney_low'] == 1 else
                                           'high' if label_row['kidney_high'] == 1 else None)
                            }
                            # Only proceed if the organ's label is not None
                            if label_info[label] is not None:
                                combined_label = f"{label}_{label_info[label]}" 
                                # Assign class ID based on combined labels mapping
                                if combined_label in combined_labels:
                                    cls_id = combined_labels[combined_label]
                                    x_center = (x + w / 2) / resized_img.shape[1]
                                    y_center = (y + h / 2) / resized_img.shape[0]
                                    width = w / resized_img.shape[1]
                                    height = h / resized_img.shape[0]
                                    f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
                                else:
                                    print(f"Warning: Combined label '{combined_label}' not found in combined_labels. Skipping annotation...")
                            else:
                                print(f"Warning: Organ '{label}' is not present in the mask for {combined_id}. Skipping annotation...")
        print(f"Total slices processed for {organ_name}: {data.shape[2]}")
scale_factor=51
# segmentation_dir= '../Inputs/organs_masks'
mask_dir='../Outputs/Validation/Segmentor_Model/masks'
annotation_dir='../Outputs/Validation/Segmentor_Model/annotation'
train_merged='../Outputs/Preprocessing/merged_dataframe.csv'
label_df = pd.read_csv(train_merged)


In [8]:
"""
The code imports a YAML configuration file containing several YOLO models associated with colon, kidney, liver, and spleen. 
The function extracts particular parameters from the setup and assigns a data split ratio of 0.8.
"""
def load_model_config(model_config_path):
    """Load model YAML configuration file."""
    with open(model_config_path, 'r') as file:
        return yaml.safe_load(file)
model_config_path = os.path.join('../config', 'segmentor_model','model_seg.yaml')
model_config_path = model_config_path.replace("\\", "/")
model_config = load_model_config(model_config_path)
bowel_seg = model_config['bowel_seg']
kidney_seg = model_config['kidney_seg']
liver_seg = model_config['liver_seg']
spleen_seg = model_config['spleen_seg']
split_ratio=0.8

# Training & validation - Bowel Model 

In [10]:
combined_labels = {
    'bowel_healthy': 0,
    'bowel_injury': 1}
segmentation_dir='../Inputs/organs_masks/bowel'
make_masks_bounding_box_and_annotations(scale_factor, segmentation_dir, mask_dir, annotation_dir, label_df, combined_labels)
data_split_train_valid(bowel_seg['source_img_dir'], bowel_seg['source_label_dir'], bowel_seg['dataset_dir'], split_ratio)
yolo_model(bowel_seg['results_dir'], bowel_seg['yaml_file'], bowel_seg['model_name'],bowel_seg['epoch'], bowel_seg['image_size'],bowel_seg['batch_size'])


Processing file: ../Inputs/organs_masks/bowel/bowel_10217_16066.nii
.............Processing..........:   bowel_10217_16066.nii
Shape of Image: (512, 512, 690)
Total slices processed for bowel: 690

Processing file: ../Inputs/organs_masks/bowel/bowel_10217_16066_0.nii
.............Processing..........:   bowel_10217_16066_0.nii
Shape of Image: (512, 512, 690)
Total slices processed for bowel: 690

Processing file: ../Inputs/organs_masks/bowel/bowel_10228_40471.nii
.............Processing..........:   bowel_10228_40471.nii
Shape of Image: (512, 512, 391)
Total slices processed for bowel: 391

Processing file: ../Inputs/organs_masks/bowel/bowel_10228_40471_0.nii
.............Processing..........:   bowel_10228_40471_0.nii
Shape of Image: (512, 512, 391)
Total slices processed for bowel: 391

Processing file: ../Inputs/organs_masks/bowel/bowel_10300_31085.nii
.............Processing..........:   bowel_10300_31085.nii
Shape of Image: (512, 512, 171)
Total slices processed for bowel: 171

P

train: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\bowel\datasets\data\train\labels... 37211 images, 0 backgrounds, 0 corrupt: 100%|██████████| 37211/37211 [02:05<00:00, 297.66it/s]


train: New cache created: E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\bowel\datasets\data\train\labels.cache


val: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\bowel\datasets\data\valid\labels... 9303 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9303/9303 [00:36<00:00, 255.26it/s]


val: New cache created: E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\bowel\datasets\data\valid\labels.cache
Plotting labels to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       9303      23353      0.833      0.731      0.809      0.804

50 epochs completed in 7.425 hours.
Optimizer stripped from E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train2\weights\last.pt, 6.2MB
Optimizer stripped from E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train2\weights\best.pt, 6.2MB

Validating E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train2\weights\best.pt...
Ultralytics YOLOv8.2.60  Python-3.9.19 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 582/582 [01:03<00:00,  9.22it/s]


                   all       9303      23353      0.832      0.731      0.809      0.804
         bowel_healthy       8302      20711      0.915      0.968      0.963      0.959
          bowel_injury       1001       2642      0.748      0.493      0.654      0.649
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train2
Training completed.
Training Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000028700287E20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,  

val: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\bowel\datasets\data\valid\labels.cache... 9303 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9303/9303 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 582/582 [01:15<00:00,  7.68it/s]


                   all       9303      23353      0.832      0.731      0.809      0.804
         bowel_healthy       8302      20711      0.915      0.968      0.964      0.959
          bowel_injury       1001       2642      0.749      0.494      0.654      0.649
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\val2
Validation completed.
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002872DC244F0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,

# Training & validation - Kidney Model

In [11]:
combined_labels = {
    'kidney_healthy': 0,
    'kidney_low': 1,
    'kidney_high': 2
     }
segmentation_dir='../Inputs/organs_masks/kidney'
make_masks_bounding_box_and_annotations(scale_factor, segmentation_dir, mask_dir, annotation_dir, label_df, combined_labels)
data_split_train_valid(kidney_seg['source_img_dir'], kidney_seg['source_label_dir'], kidney_seg['dataset_dir'], split_ratio)
yolo_model(kidney_seg['results_dir'], kidney_seg['yaml_file'], kidney_seg['model_name'],kidney_seg['epoch'], kidney_seg['image_size'], 
           kidney_seg['batch_size'])


Processing file: ../Inputs/organs_masks/kidney/kidney_10217_16066.nii
.............Processing..........:   kidney_10217_16066.nii
Shape of Image: (512, 512, 690)
Total slices processed for kidney: 690

Processing file: ../Inputs/organs_masks/kidney/kidney_10217_16066_0.nii
.............Processing..........:   kidney_10217_16066_0.nii
Shape of Image: (512, 512, 690)
Total slices processed for kidney: 690

Processing file: ../Inputs/organs_masks/kidney/kidney_10228_40471.nii
.............Processing..........:   kidney_10228_40471.nii
Shape of Image: (512, 512, 391)
Total slices processed for kidney: 391

Processing file: ../Inputs/organs_masks/kidney/kidney_10228_40471_0.nii
.............Processing..........:   kidney_10228_40471_0.nii
Shape of Image: (512, 512, 391)
Total slices processed for kidney: 391

Processing file: ../Inputs/organs_masks/kidney/kidney_10300_31085.nii
.............Processing..........:   kidney_10300_31085.nii
Shape of Image: (512, 512, 171)
Total slices processe

train: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\kidney\datasets\data\train\labels... 14968 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14968/14968 [00:41<00:00, 360.06it/s]


train: New cache created: E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\kidney\datasets\data\train\labels.cache


val: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\kidney\datasets\data\valid\labels... 3743 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3743/3743 [00:11<00:00, 316.87it/s]


val: New cache created: E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\kidney\datasets\data\valid\labels.cache
Plotting labels to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


50 epochs completed in 3.762 hours.
Optimizer stripped from E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train3\weights\last.pt, 6.2MB
Optimizer stripped from E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train3\weights\best.pt, 6.2MB

Validating E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train3\weights\best.pt...
Ultralytics YOLOv8.2.60  Python-3.9.19 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 234/234 [00:23<00:00,  9.88it/s]


                   all       3743       3774      0.705      0.719      0.802      0.801
        kidney_healthy       1780       1791      0.694      0.901      0.859      0.859
            kidney_low       1198       1201      0.727      0.661      0.807      0.806
           kidney_high        765        782      0.693      0.596       0.74      0.738
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train3
Training completed.
Training Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000286FFCD9B50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004, 

val: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\kidney\datasets\data\valid\labels.cache... 3743 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3743/3743 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 234/234 [00:29<00:00,  7.84it/s]


                   all       3743       3774      0.704      0.722      0.802      0.801
        kidney_healthy       1780       1791      0.694      0.902      0.859      0.859
            kidney_low       1198       1201      0.726      0.662      0.807      0.806
           kidney_high        765        782      0.693      0.602      0.741      0.738
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\val3
Validation completed.
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000286F20029D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004

# Training & validation - Liver Model 

In [12]:
combined_labels = {
    'liver_healthy': 0,
    'liver_low': 1,
    'liver_high': 2
     }
segmentation_dir='../Inputs/organs_masks/liver'
make_masks_bounding_box_and_annotations(scale_factor, segmentation_dir, mask_dir, annotation_dir, label_df, combined_labels)
data_split_train_valid(liver_seg['source_img_dir'], liver_seg['source_label_dir'], liver_seg['dataset_dir'], split_ratio)
yolo_model(liver_seg['results_dir'], liver_seg['yaml_file'], liver_seg['model_name'],liver_seg['epoch'], liver_seg['image_size'],liver_seg['batch_size'])


Processing file: ../Inputs/organs_masks/liver/liver_10217_16066.nii
.............Processing..........:   liver_10217_16066.nii
Shape of Image: (512, 512, 690)
Total slices processed for liver: 690

Processing file: ../Inputs/organs_masks/liver/liver_10228_40471.nii
.............Processing..........:   liver_10228_40471.nii
Shape of Image: (512, 512, 391)
Total slices processed for liver: 391

Processing file: ../Inputs/organs_masks/liver/liver_10300_31085.nii
.............Processing..........:   liver_10300_31085.nii
Shape of Image: (512, 512, 171)
Total slices processed for liver: 171

Processing file: ../Inputs/organs_masks/liver/liver_10937_53000.nii
.............Processing..........:   liver_10937_53000.nii
Shape of Image: (512, 512, 221)
Total slices processed for liver: 221

Processing file: ../Inputs/organs_masks/liver/liver_11177_13041.nii
.............Processing..........:   liver_11177_13041.nii
Shape of Image: (512, 512, 375)
Total slices processed for liver: 375

Processin

train: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\liver\datasets\data\train\labels... 12884 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12884/12884 [00:44<00:00, 287.85it/s]


train: New cache created: E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\liver\datasets\data\train\labels.cache


val: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\liver\datasets\data\valid\labels... 3222 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3222/3222 [00:12<00:00, 267.87it/s]


val: New cache created: E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\liver\datasets\data\valid\labels.cache
Plotting labels to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train4
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


50 epochs completed in 2.384 hours.
Optimizer stripped from E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train4\weights\last.pt, 6.2MB
Optimizer stripped from E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train4\weights\best.pt, 6.2MB

Validating E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train4\weights\best.pt...
Ultralytics YOLOv8.2.60  Python-3.9.19 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 202/202 [00:20<00:00,  9.63it/s]


                   all       3222       3973       0.88      0.806        0.9       0.89
         liver_healthy       1310       1649      0.919      0.847      0.937      0.927
             liver_low       1603       1938      0.904      0.904      0.959      0.952
            liver_high        309        386      0.816      0.667      0.803      0.791
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train4
Training completed.
Training Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000286FE559970>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004, 

val: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\liver\datasets\data\valid\labels.cache... 3222 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3222/3222 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 202/202 [00:26<00:00,  7.57it/s]


                   all       3222       3973      0.879      0.806        0.9       0.89
         liver_healthy       1310       1649      0.918      0.847      0.937      0.927
             liver_low       1603       1938      0.903      0.904      0.959      0.951
            liver_high        309        386      0.815      0.666      0.803      0.791
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\val4
Validation completed.
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000286FFD2DB80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004

# Training & validation - Spleen Model

In [13]:
combined_labels = {
    'spleen_healthy': 0,
    'spleen_low': 1,
    'spleen_high': 2
     }
segmentation_dir='../Inputs/organs_masks/spleen'
make_masks_bounding_box_and_annotations(scale_factor, segmentation_dir, mask_dir, annotation_dir, label_df, combined_labels)
data_split_train_valid(spleen_seg['source_img_dir'], spleen_seg['source_label_dir'], spleen_seg['dataset_dir'], split_ratio)
yolo_model(spleen_seg['results_dir'], spleen_seg['yaml_file'], spleen_seg['model_name'],spleen_seg['epoch'], spleen_seg['image_size'], spleen_seg['batch_size'])


Processing file: ../Inputs/organs_masks/spleen/spleen_10217_16066.nii
.............Processing..........:   spleen_10217_16066.nii
Shape of Image: (512, 512, 690)
Total slices processed for spleen: 690

Processing file: ../Inputs/organs_masks/spleen/spleen_10228_40471.nii
.............Processing..........:   spleen_10228_40471.nii
Shape of Image: (512, 512, 391)
Total slices processed for spleen: 391

Processing file: ../Inputs/organs_masks/spleen/spleen_10300_31085.nii
.............Processing..........:   spleen_10300_31085.nii
Shape of Image: (512, 512, 171)
Total slices processed for spleen: 171

Processing file: ../Inputs/organs_masks/spleen/spleen_10937_53000.nii
.............Processing..........:   spleen_10937_53000.nii
Shape of Image: (512, 512, 221)
Total slices processed for spleen: 221

Processing file: ../Inputs/organs_masks/spleen/spleen_11177_13041.nii
.............Processing..........:   spleen_11177_13041.nii
Shape of Image: (512, 512, 375)
Total slices processed for sp

train: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\spleen\datasets\data\train\labels... 7856 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7856/7856 [00:27<00:00, 290.64it/s]


train: New cache created: E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\spleen\datasets\data\train\labels.cache


val: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\spleen\datasets\data\valid\labels... 1965 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1965/1965 [00:07<00:00, 272.18it/s]


val: New cache created: E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\spleen\datasets\data\valid\labels.cache
Plotting labels to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train5
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


50 epochs completed in 1.474 hours.
Optimizer stripped from E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train5\weights\last.pt, 6.2MB
Optimizer stripped from E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train5\weights\best.pt, 6.2MB

Validating E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train5\weights\best.pt...
Ultralytics YOLOv8.2.60  Python-3.9.19 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 123/123 [00:12<00:00,  9.53it/s]


                   all       1965       2152      0.881      0.824      0.923      0.916
        spleen_healthy        554        617      0.851      0.786      0.889       0.88
            spleen_low        469        490      0.881      0.797      0.922      0.918
           spleen_high        942       1045      0.912       0.89      0.959      0.949
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\train5
Training completed.
Training Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000286D22BDB80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004, 

val: Scanning E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Validation\Segmentor_Model\modelling\spleen\datasets\data\valid\labels.cache... 1965 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1965/1965 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 123/123 [00:16<00:00,  7.37it/s]


                   all       1965       2152      0.883      0.824      0.923      0.915
        spleen_healthy        554        617      0.851      0.787      0.889      0.879
            spleen_low        469        490      0.883      0.794      0.922      0.918
           spleen_high        942       1045      0.914       0.89      0.959      0.949
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to E:\MS_AI_ML\Dissertation\Abdominal\Outputs\Preprocessing\Results\run\detect\val5
Validation completed.
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002870020E640>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004